# Example notebook for the functions contained in CRYSTALpytools.crystal_io

### Crystal_input class

In [1]:
from CRYSTALpytools.crystal_io import Crystal_input

#### Create a crystal input object by directly calling methods

In [2]:
mgo_input = Crystal_input()
mgo_input.geom.title('MGO BULK - GEOMETRY TEST')
mgo_input.geom.crystal(225, # Space group
                      [4.217], # Minimal set of lattice parameters
                      [[12, 0., 0., 0.], # Atomic labels and coordinates
                       [8, 0.5, 0.5, 0.5]]
                      )
mgo_input.basisset.basisset('POB-DZVP')
mgo_input.scf.dft.xcfunc('B3LYP')
mgo_input.scf.dft.xxlgrid()
mgo_input.scf.tolinteg(7, 7, 7, 7, 14)
mgo_input.scf.shrink(12, 24)
mgo_input.scf.maxcycle(70)
mgo_input.scf.fmixing(70)
mgo_input.scf.diis()

print(mgo_input.data)

MGO BULK - GEOMETRY TEST
CRYSTAL
0   0   0   
225 
4.217000     
2 
12     0.00000000   0.00000000   0.00000000 
8      0.50000000   0.50000000   0.50000000 
BASISSET
POB-DZVP 
DFT
B3LYP
XXLGRID
ENDDFT
MAXCYCLE
70 
SHRINK
12 24 
TOLINTEG
7 7 7 7 14 
FMIXING
70 
DIIS
ENDSCF



#### Create a crystal input object from blocks
'Block' is a multiple-line string closed by the 'END' keyword. 3 blocks are defined in CRYSTAL d12 format: geometry, basis set and SCF. There are sub-blocks in the 3 major blocks that can be defined in a similar way by calling corresponding keywords.

- To set value for keywords,  use `obj.keyword(value)`.

In [3]:
geom_block = \
"""MGO BULK - GEOMETRY TEST
CRYSTAL
0 0 0
225
4.217
2
12 0.    0.    0.
8 0.5   0.5   0.5
ENDGEOM
"""

bs_block   = \
"""BASISSET
POB-DZVP
"""
func_block = \
"""DFT
B3LYP
XXLGRID
ENDDFT
"""
scf_block  = \
"""TOLINTEG
7 7 7 7 14
SHRINK
12 24
MAXCYCLE
200
FMIXING
70
DIIS
ENDSCF
"""
mgo_input = Crystal_input()
mgo_input.geom(geom_block)
mgo_input.basisset(bs_block)
mgo_input.scf(func_block + scf_block) # DFT is a sub-block of SCF block
print(mgo_input.data)

# Clean entries in SCF block and reset DFT subblock only
mgo_input.scf()
mgo_input.scf.dft(func_block)
print(mgo_input.data)

MGO BULK - GEOMETRY TEST
CRYSTAL
0 0 0
225
4.217
2
12 0.    0.    0.
8 0.5   0.5   0.5
BASISSET
POB-DZVP
DFT
B3LYP
XXLGRID
ENDDFT
MAXCYCLE
200
SHRINK
12 24
TOLINTEG
7 7 7 7 14
FMIXING
70
DIIS
ENDSCF

MGO BULK - GEOMETRY TEST
CRYSTAL
0 0 0
225
4.217
2
12 0.    0.    0.
8 0.5   0.5   0.5
BASISSET
POB-DZVP
DFT
B3LYP
XXLGRID
ENDDFT
ENDSCF



#### Create a crystal input object from an existing input file

The existing file can act as a template, which is read during initialization. By calling the corresponding method one can change, delete and substitute keywords.

- To edite the keyword, the corresponding block \- subblock \- keyword method should be called and the new values should be used as the input. 

- To delete values of the keyword or all the entries in blocks, use `''`. `''` can remove everything and leave the keyword only. For example, `obj.geom.optgeom('')` will get 'OPTGEOM\nENDOPT'.

- To remove everything, use `None`. For example, `obj.geom.optgeom(None)` will completely remove everything. But if `obj.geom.optgeom` attribute or method is called afterwards, it is initialized and generates a 'OPTGEOM\nENDOPT' string


In [4]:
mgo_input = Crystal_input('data/mgo.d12')
# Set Optgeom block
mgo_input.geom.optgeom.toldex(0.0012)
mgo_input.geom.optgeom.toldeg(0.0003)
print(mgo_input.geom.data)
# Delete OPTGEOM command
mgo_input.geom.optgeom(None)
print(mgo_input.geom.data)
# Write the updated object to file
mgo_input.write_file('data/mgo_inputtest.d12')

Generated by CRYSTALpytools
MGO BULK - GEOMETRY TEST
CRYSTAL
0 0 0
225
4.217
2
12 0.    0.    0.
8 0.5   0.5   0.5
OPTGEOM
TOLDEG
0.0003 
TOLDEX
0.0012 
ENDOPT
ENDGEOM

Generated by CRYSTALpytools
MGO BULK - GEOMETRY TEST
CRYSTAL
0 0 0
225
4.217
2
12 0.    0.    0.
8 0.5   0.5   0.5
ENDGEOM



#### Set geometry

The user can set the geometry input by a CIF file or a pymatgen structure when needed. There are 2 available options:

1. Use the 'EXTERNAL' keyword and `Crystal_input` object automatically convert the geometry entry to a gui file with default settings (recommended)  
2. Use the 'CRYSTAL' keyword. Limited to 3D structures.

In [5]:
paracetamol = Crystal_input()
paracetamol.geom_from_cif('data/paracetamol.cif', keyword='CRYSTAL')
print(paracetamol.geom.data)
paracetamol = Crystal_input().geom_from_cif('data/paracetamol.cif',
                                            gui_name='data/paracetamol.gui')

Generated by CRYSTALpytools
CRYSTAL
0   0   0   
14  
7.073000     9.166000     12.667000    115.510000   
20 
1      0.37160000   0.92980000   0.49880000 
1      0.24220000   0.76340000   0.33040000 
1      0.77660000   0.60000000   0.43090000 
1      0.90770000   0.76640000   0.60050000 
1      0.79990000   0.53530000   0.25760000 
1      0.10820000   0.95930000   0.69510000 
1      0.56840000   0.08530000   0.89610000 
1      0.29390000   0.07980000   0.85010000 
1      0.40860000   0.21910000   0.80230000 
6      0.14960000   0.85790000   0.56187000 
6      0.24320000   0.85758000   0.48526000 
6      0.17040000   0.76206000   0.39004000 
6      0.00430000   0.66838000   0.37078000 
6      0.90790000   0.67092000   0.44601000 
6      0.98060000   0.76560000   0.54106000 
6      0.39830000   0.01599000   0.71975000 
6      0.41680000   0.10369000   0.82440000 
7      0.21229000   0.94984000   0.66089000 
8      0.94130000   0.57511000   0.27811000 
8      0.54450000   0.00741000   0

It is strongly recommended to set conventional atomic numbers by calling methods such as `Crystal_input.geom.crystal()` and manually type corresponding atomic numbers. Even though `zconv` entry can be provided when using `Crystal_input.geom_from_cif` and `Crystal_input.geom_from_pmg`, sequences of atoms in the input file might not be consistent with sequences in output.

#### Set basis set

The user can set basis set by string (as shown before), a text file or downloading it from [Basis Set Exchange (BSE)](https://www.basissetexchange.org/). Besides, a built-in BS can be called by keyword.

To download BSs from BSE, the name of basis set and conventional atomic numbers of elements are needed. 

Note that in BSE, the charge information is missing. A neutral atom is always assumed by default, which works well in most cases. To modify the initial charge guess, see below. 

Automatic assignment of charges is available only for all-electron BSs.

In [6]:
# This is a wrapper to quickly set basis sets.
# Equivalent to `mgo_input.basisset.from_bse('6-311G*', [12, 8])`
mgo_input.bs_user('6-31G*', z=[12, 8])
print(mgo_input.basisset.data)

12 5
0 0 6 2.00 1.00
  11722.8000000000      0.0019778293
   1759.9300000000      0.0151139948
    400.8460000000      0.0739107745
    112.8070000000      0.2491909140
     35.9997000000      0.4879278316
     12.1828000000      0.3196618896
0 1 6 8.00 1.00
    189.1800000000     -0.0032371705      0.0049281299
     45.2119000000     -0.0410079060      0.0349887994
     14.3563000000     -0.1126000164      0.1407249977
      5.1388600000      0.1486330216      0.3336419947
      1.9065200000      0.6164970898      0.4449399929
      0.7058870000      0.3648290531      0.2692539957
0 1 3 2.00 1.00
      0.9293400000     -0.2122908985     -0.0224191812
      0.2690350000     -0.1079854570      0.1922708390
      0.1173790000      1.1758449770      0.8461802916
0 1 1 0.00 1.00
      0.0421061000      1.0000000000      1.0000000000
0 3 1 0.00 1.00
      0.1750000000      1.0000000000
8 4
0 0 6 2.00 1.00
   5484.6716600000      0.0018310744
    825.2349460000      0.0139501722
    188.0469

In [7]:
# Read a Gaussian STO-3G basis set
# Equivalent to `mgo_input.basisset.from_file('data/mgo_inputbs.gbs', 'gaussian94')`
mgo_input.bs_user('data/mgo_inputbs.gbs', fmt='gaussian94')
print(mgo_input.basisset.data)

8 2
0 0 3 2.00 1.00
    130.7093214000      0.1543289673
     23.8088660500      0.5353281423
      6.4436083130      0.4446345422
0 1 3 6.00 1.00
      5.0331513190     -0.0999672292      0.1559162750
      1.1695961250      0.3995128261      0.6076837186
      0.3803889600      0.7001154689      0.3919573931
12 3
0 0 3 2.00 1.00
    299.2374137000      0.1543289673
     54.5064684500      0.5353281423
     14.7515775200      0.4446345422
0 1 3 8.00 1.00
     15.1218235200     -0.0999672292      0.1559162750
      3.5139865790      0.3995128261      0.6076837186
      1.1428574980      0.7001154689      0.3919573931
0 1 3 2.00 1.00
      1.3954482930     -0.2196203690      0.0105876043
      0.3893265318      0.2255954336      0.5951670053
      0.1523797659      0.9003984260      0.4620010120
99 0
ENDBS



In some rare cases, the user might want to change the charge of BS shells to get ions and a better initial guess of charge density. A dictionary is used to define the charge.

- Key: Conventional atomic number
- Value: nshell\*1 list of charges. Must be consistent with BS definitions.

In [8]:
# O2- and Mg2+ configuration.
chg = {8  : [2.0, 8.0],
       12 : [2.0, 8.0, 0.0]}
mgo_input.bs_user('data/mgo_inputbs.gbs', fmt='gaussian94', charge=chg)
print(mgo_input.basisset.data)

8 2
0 0 3 2.00 1.00
    130.7093214000      0.1543289673
     23.8088660500      0.5353281423
      6.4436083130      0.4446345422
0 1 3 8.00 1.00
      5.0331513190     -0.0999672292      0.1559162750
      1.1695961250      0.3995128261      0.6076837186
      0.3803889600      0.7001154689      0.3919573931
12 3
0 0 3 2.00 1.00
    299.2374137000      0.1543289673
     54.5064684500      0.5353281423
     14.7515775200      0.4446345422
0 1 3 8.00 1.00
     15.1218235200     -0.0999672292      0.1559162750
      3.5139865790      0.3995128261      0.6076837186
      1.1428574980      0.7001154689      0.3919573931
0 1 3 0.00 1.00
      1.3954482930     -0.2196203690      0.0105876043
      0.3893265318      0.2255954336      0.5951670053
      0.1523797659      0.9003984260      0.4620010120
99 0
ENDBS



Similarly, effective core pseudopotential can be defined. The following example uses Stuttgart ECP.

Note that the automatic assignment of atomic charge is not available for ECP BSs. `charge` keyword is needed.

In [9]:
ECP = {308 : 'STUTSC',
       312 : 'STUTSC'}
chg = {308 : [0., 2., 4., 0.],
       312 : [0., 2., 0., 0.]}
mgo_input.bs_user('Stuttgart RLC ECP', z=[308, 312], ECP=ECP, charge=chg)
print(mgo_input.basisset.data)

308 4
STUTSC
0 4 1 0.00 1.00
      1.0000000000      0.0000000000
0 0 1 2.00 1.00
     10.4456700000     50.7710690000
0 2 1 4.00 1.00
     18.0451740000     -4.9035510000
0 3 1 0.00 1.00
      8.1647980000     -3.3121240000
312 4
STUTSC
0 4 1 0.00 1.00
      1.0000000000      0.0000000000
0 0 1 2.00 1.00
      1.7320000000     14.6760000000
0 2 1 0.00 1.00
      1.1150000000      5.1757000000
0 3 1 0.00 1.00
      1.2030000000     -1.8160000000
99 0
ENDBS



/home/huanyu/apps/anaconda3/envs/crystal_py3.9/lib/python3.9/site-packages/CRYSTALpytools/base/basisset.py:299: UserWarning: ECP is used. Automatic assignment of atomic charge is disabled.
  self.atoms.append(AtomBS.read_bse(bs, onez))


To set conventional atomic numbers for basis sets, the user can use `bs_user()` twice with `append=True`.

In [10]:
obj = Crystal_input()
obj.basisset.from_bse('6-31G*', [6, 1]) # Normal definition
obj.basisset.from_bse('def2-SVP', [101], append=True) # Define a second basis set for H with label 101
print(obj.basisset.data)

6 4
0 0 6 2.00 1.00
   3047.5248800000      0.0018347371
    457.3695180000      0.0140373228
    103.9486850000      0.0688426223
     29.2101553000      0.2321844432
      9.2866629600      0.4679413484
      3.1639269600      0.3623119853
0 1 3 4.00 1.00
      7.8682723500     -0.1193324198      0.0689990666
      1.8812885400     -0.1608541517      0.3164239610
      0.5442492580      1.1434564380      0.7443082909
0 1 1 0.00 1.00
      0.1687144782      1.0000000000      1.0000000000
0 3 1 0.00 1.00
      0.8000000000      1.0000000000
1 2
0 0 3 1.00 1.00
     18.7311369600      0.0334946043
      2.8253943650      0.2347269535
      0.6401216923      0.8137573261
0 0 1 0.00 1.00
      0.1612777588      1.0000000000
101 3
0 0 3 1.00 1.00
     13.0107010000      0.0196821580
      1.9622572000      0.1379652400
      0.4445379600      0.4783193500
0 0 1 0.00 1.00
      0.1219496200      1.0000000000
0 2 1 0.00 1.00
      0.8000000000      1.0000000000
99 0
ENDBS



In principle, to use 'BASISSET' keyword, `mgo_input.basisset.basisset()` should be called. A shortcut is added.

In [11]:
mgo_input.bs_keyword('pob-DZVP')
print(mgo_input.data)

Generated by CRYSTALpytools
MGO BULK - GEOMETRY TEST
CRYSTAL
0 0 0
225
4.217
2
12 0.    0.    0.
8 0.5   0.5   0.5
BASISSET
pob-DZVP 
DFT
B3LYP
XXLGRID
ENDDFT
MAXCYCLE
200
SHRINK
12 24
TOLINTEG
7 7 7 7 14
FMIXING
70
DIIS
ENDSCF



/home/huanyu/apps/anaconda3/envs/crystal_py3.9/lib/python3.9/site-packages/CRYSTALpytools/crystal_io.py:226: UserWarning: User's definition of baisis set is not empty. It will be covered by 'BASISSET' keyword.
  self.basisset.basisset(keyword)


### Crystal_output class

In [1]:
from CRYSTALpytools.crystal_io import Crystal_output

#### Read output case 1: Periodic system

In [2]:
mgo_output = Crystal_output().read_file('data/mgo_optgeom.out')
mgo_output

- Substracting basice energy and electronic information

In [3]:
# Initial SCF energy
print("Final energy = %s eV \n" % mgo_output.get_final_energy())
# Initial SCF energy, an alternative way
mgo_output.get_convergence(history=False)
print("Final energy = %s eV \n" % mgo_output.final_energy)

# Initial SCF Fermi energy
print("Fermi energy = %s eV \n" % mgo_output.get_fermi_energy())

# Initial SCF Band gap
print("Band gap = %s eV \n" % mgo_output.get_band_gap())

# Mulliken charge, Note that is based on the optimized geometry
print("Mulliken charge (e) = ", mgo_output.get_mulliken_charges())

Final energy = -7495.340914150903 eV 

Final energy = -7495.340914150903 eV 

Fermi energy = -4.136710311917215 eV 

Band gap = 7.1237 eV 

Mulliken charge (e) =  [10.129  9.871]


- Substracting SCF convergence

In [4]:
mgo_output.get_scf_convergence()
# Convergence status
print('SCF convergence status = {}\n'.format(mgo_output.scf_status))
print('SCF number of cycles = {:d}\n'.format(mgo_output.scf_cycles))

# Energy iteration history
print('Total energy (eV) = \n', mgo_output.scf_energy, '\n')
print('Energy difference (eV)= \n', mgo_output.scf_deltae, '\n')

# Fermi level iteration history
print('Fermi level (eV) = \n', mgo_output.get_fermi_energy(history=True), '\n')

# Band gap iteration history
print('Band gap (eV) = \n', mgo_output.get_band_gap(history=True), '\n')

SCF convergence status = converged

SCF number of cycles = 8

Total energy (eV) = 
 [-7476.82733948 -7489.97945008 -7494.91919331 -7495.33640687
 -7495.33787794 -7495.33800249 -7495.33800425 -7495.33800435] 

Energy difference (eV)= 
 [-7.48313122e+03 -1.31430996e+01 -4.95247230e+00 -4.16334210e-01
 -1.47213600e-03 -1.24628149e-04 -1.75513441e-06 -1.02586926e-07] 

Fermi level (eV) = 
 [-8.24883759 -4.59365942 -4.10322534 -4.12058103 -4.14111121 -4.13676256
 -4.13671031] 

Band gap (eV) = 
 [8.4379 6.6125 7.0523 7.1034 7.1228 7.1235 7.1237] 



- Substracting forces

Note that the forces computed at the initial SCF step is read. To read the final forces, set `initial=False` and use 'ONELOG' keyword in CRYSTAL input file. `grad=True` is valid for optimizations only.

In [5]:
mgo_output.get_forces(initial=True, grad=True)

print('Unit: Hartree/Bohr\n')
print("Forces on cell = \n %s \n\n Forces on atoms = \n %s \n\n RMS Gradient = \n %s \n\n" %
      (mgo_output.forces_cell, mgo_output.forces_atoms, mgo_output.opt_rmsgrad))

Unit: Hartree/Bohr

Forces on cell = 
 [[-0.00148273  0.00148273  0.00148273]
 [ 0.00148273 -0.00148273  0.00148273]
 [ 0.00148273  0.00148273 -0.00148273]] 

 Forces on atoms = 
 [[-4.18889278e-16 -4.11372640e-16 -4.14981546e-16]
 [ 4.14263349e-16  4.13107364e-16  4.14981546e-16]] 

 RMS Gradient = 
 [2.0466e-02 4.6930e-03 6.9000e-05 3.2000e-05] 




- Substracting basic geometry information

Option 1: From `get_*` methods

By setting `initial=False` the optimized geometry can be printed. In other cases such as SCF calculations, `initial=False` has no effect but takes slightly longer to get results

In [6]:
# Symmetry operators
print("Symmetry operators = \n %s \n" % mgo_output.get_symmops())

# Pre-optimized geometry
print('Pre-optimized geometry\n', mgo_output.get_geometry(initial=True), '\n')

# Optimized geometry
print('Optimized geometry\n', mgo_output.get_geometry(initial=False), '\n')
# Optimized geometry, an alternative, which returns a list of key informations
print('Using get_last_geom()\n', mgo_output.get_last_geom(write_gui_file=False), '\n')

# Lattice matrix
print('Lattice matrix:\n', mgo_output.get_lattice(initial=True), '\n')

# Reprocal lattice matrix
print('Reprocal lattice matrix\n', mgo_output.get_reciprocal_lattice(initial=True), '\n')

# Cell expansion matrix
print('Cell expansion matrix\n', mgo_output.get_trans_matrix(), '\n')

# Primitive optimized geometry (If supercell expansion is applied)
print('Primitive optimized geometry\n', mgo_output.get_primitive_geometry(initial=False), '\n')

# Primitive lattice matrix (If supercell expansion is applied)
print('Primitive lattice matrix\n', mgo_output.get_primitive_lattice(initial=True), '\n')

# Primitive reciprocal lattice matrix (If supercell expansion is applied)
print('Primitive reciprocal lattice matrix\n', mgo_output.get_primitive_reciprocal_lattice(initial=True), '\n')

Symmetry operators = 
 [[[ 1.  0.  0.]
  [ 0.  1.  0.]
  [ 0.  0.  1.]
  [ 0.  0.  0.]]

 [[ 0.  1.  0.]
  [ 1.  0.  0.]
  [-1. -1. -1.]
  [ 0.  0.  0.]]

 [[-1. -1. -1.]
  [ 0.  0.  1.]
  [ 0.  1.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  1.]
  [-1. -1. -1.]
  [ 1.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  1.]
  [ 1.  0.  0.]
  [ 0.  1.  0.]
  [ 0.  0.  0.]]

 [[ 0.  1.  0.]
  [ 0.  0.  1.]
  [ 1.  0.  0.]
  [ 0.  0.  0.]]

 [[ 1.  0.  0.]
  [ 0.  0.  1.]
  [-1. -1. -1.]
  [ 0.  0.  0.]]

 [[ 1.  0.  0.]
  [-1. -1. -1.]
  [ 0.  1.  0.]
  [ 0.  0.  0.]]

 [[-1. -1. -1.]
  [ 0.  1.  0.]
  [ 1.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  1.]
  [ 0.  1.  0.]
  [-1. -1. -1.]
  [ 0.  0.  0.]]

 [[ 0.  1.  0.]
  [-1. -1. -1.]
  [ 0.  0.  1.]
  [ 0.  0.  0.]]

 [[-1. -1. -1.]
  [ 1.  0.  0.]
  [ 0.  0.  1.]
  [ 0.  0.  0.]]

 [[ 0. -1.  0.]
  [-1.  0.  0.]
  [ 0.  0. -1.]
  [ 0.  0.  0.]]

 [[-1.  0.  0.]
  [ 0. -1.  0.]
  [ 1.  1.  1.]
  [ 0.  0.  0.]]

 [[ 0.  0. -1.]
  [ 1.  1.  1.]
  [ 0. -1.  0.]
  [ 0

Option 2: Basic properties can be directly called as properties

For atomic coordinates, that is equivalent to `get_geometry(initial=True)`, i.e., the initial geometry.

In [7]:
# Space group number and symbol
print('Space group = {:d} {:s}\n'.format(mgo_output.sg_number, mgo_output.sg_symbol))
# Number of atoms
print('Number of atoms = {:d}\n'.format(mgo_output.n_atoms))
# Atomic symbols
print('Species = ', mgo_output.atom_symbols, '\n')
# Conventional atomic numbers, consistent with CRYSTAL (e.g., 101 for H with pesudopotentials)
print('Conventional atomic numbers = ', mgo_output.atom_numbers, '\n')
# Atomic positions, consistent with CRYSTAL (e.g., Fractional, Fractional, Cartesian for 2D periodic systems)
print('Atomic positions = \n', mgo_output.atom_positions, '\n')
# Fractional atomic positions
print('Fractional atomic positions = \n', mgo_output.atom_positions_frac, '\n')
# Cartesian atomic positions
print('Cartesian atomic positions = \n', mgo_output.atom_positions_cart, '\n')

Space group = 225 F M 3 M

Number of atoms = 2

Species =  ['Mg', 'O'] 

Conventional atomic numbers =  [12  8] 

Atomic positions = 
 [[0.  0.  0. ]
 [0.5 0.5 0.5]] 

Fractional atomic positions = 
 [[0.  0.  0. ]
 [0.5 0.5 0.5]] 

Cartesian atomic positions = 
 [[ 0.          0.          0.        ]
 [-2.08606614  2.32541051  1.89158949]] 



- Substracting Optimization convergence and trajectory

Trajectory is written in 'data/mgo_optgeom.opttraj' in CRYSTAL gui format.

In [8]:
!mkdir data/mgo_optgeom.opttraj
mgo_output.get_opt_convergence(write_gui=True, gui_name='data/mgo_optgeom.opttraj/mgo')

print('Optimization convergence status = {:s}\n'.format(mgo_output.opt_status))
print('Optimization cycles = {:d}\n'.format(mgo_output.opt_cycles))
print('Optimization energy history (eV) = \n', mgo_output.opt_energy, '\n')
print('Optimization energy difference (eV) = \n', mgo_output.opt_deltae, '\n')
print('Optimization max gradient (Hartree/Bohr) = \n', mgo_output.opt_maxgrad, '\n')
print('Optimization RMS gradient (Hartree/Bohr) = \n', mgo_output.opt_rmsgrad, '\n')
print('Optimization max displacement (Bohr) = \n', mgo_output.opt_maxdisp, '\n')
print('Optimization RMS displacement (Bohr) = \n', mgo_output.opt_rmsdisp, '\n')

mkdir: cannot create directory ‘data/mgo_optgeom.opttraj’: File exists
Optimization convergence status = converged

Optimization cycles = 4

Optimization energy history (eV) = 
 [ -275.44859114 -7495.3406783  -7495.34087403 -7495.34091415] 

Optimization energy difference (eV) = 
 [-2.75448591e+02 -2.67406293e-03 -1.95731501e-04 -4.01367947e-05] 

Optimization max gradient (Hartree/Bohr) = 
 [2.0466e-02 4.6930e-03 6.9000e-05 3.2000e-05] 

Optimization RMS gradient (Hartree/Bohr) = 
 [2.0466e-02 4.6930e-03 6.9000e-05 3.2000e-05] 

Optimization max displacement (Bohr) = 
 [0.000e+00 2.177e-03 3.200e-05 1.500e-05] 

Optimization RMS displacement (Bohr) = 
 [0.000e+00 2.177e-03 3.200e-05 1.500e-05] 



#### 0D system

In [10]:
co_output = Crystal_output().read_file('data/co.out')

In [11]:
# Number of atoms
print('Number of atoms = {:d}\n'.format(co_output.n_atoms))
# Atomic symbols
print('Species = ', co_output.atom_symbols, '\n')
# Atomic positions, consistent with CRYSTAL (e.g., Fractional, Fractional, Cartesian for 2D periodic systems)
print('Atomic positions = \n', co_output.atom_positions, '\n')

# Final energy
print("Final energy = %s eV \n" % co_output.get_final_energy())

# Fermi energy
print("Fermi energy = %s eV \n" % co_output.get_fermi_energy())

# Primitive lattice
print("Primitive lattice \n %s \n" % co_output.get_primitive_lattice())

# Reciprocal lattice
print("Reciprocal lattice \n %s \n" % co_output.get_reciprocal_lattice())

# Band gap
print("Band gap = %s eV \n" % co_output.get_band_gap())

# Last geometry
print("Last geometry = \n %s \n" % co_output.get_last_geom())

# Symmetry operators
print("Symmetry operators = \n %s \n" % co_output.get_symmops())

# Forces
co_output.get_forces()
print("Forces on cell = \n %s \n\n Forces on atoms = \n %s \n\n" %
      (co_output.forces_cell, co_output.forces_atoms))

# Scf convergence
co_output.get_scf_convergence()
print("SCF energy history = \n %s \n\n Energy difference = \n %s \n\n" %
      (co_output.scf_energy, co_output.scf_deltae))

Number of atoms = 2

Species =  ['C', 'O'] 

Atomic positions = 
 [[0.  0.  0. ]
 [0.8 0.5 0.4]] 

Final energy = -3040.063479187389 eV 

Fermi energy = -7.247933954451301 eV 

Primitive lattice 
 None 

Reciprocal lattice 
 None 

Band gap = 10.9012 eV 

Last geometry = 
 [[[500.0, 0.0, 0.0], [0.0, 500.0, 0.0], [0.0, 0.0, 500.0]], array([6, 8]), [[0.0, 0.0, 0.0], [0.8, 0.5, 0.4]]] 

Symmetry operators = 
 [] 

Forces on cell = 
 [] 

 Forces on atoms = 
 [[-0.48652346 -0.30407721 -0.24326191]
 [ 0.48652346  0.30407721  0.24326191]] 


SCF energy history = 
 [-3050.41709057 -3037.8127751  -3037.86317042 -3037.87780122
 -3037.87917172 -3037.87917184 -3037.87917184] 

 Energy difference = 
 [-3.04767526e+03  1.25988718e+01 -5.03410646e-02 -1.46397258e-02
 -1.37145387e-03 -1.17281075e-07 -7.48313122e-10] 




/home/huanyu/apps/anaconda3/envs/crystal_py3.9/lib/python3.9/site-packages/CRYSTALpytools/crystal_io.py:807: UserWarning: 0D system. No lattice.
  warnings.warn('0D system. No lattice.')
/home/huanyu/apps/anaconda3/envs/crystal_py3.9/lib/python3.9/site-packages/CRYSTALpytools/crystal_io.py:523: UserWarning: 0D system. No lattice.
  warnings.warn('0D system. No lattice.')


### Properties_input class

In [1]:
from CRYSTALpytools.crystal_io import Properties_input

#### Create a properties input object by directly calling methods

In [2]:
# Create the bands input object
inp = Properties_input()
# NEWK + ECHG plot
inp.newk(12, 24, 1, 0)
inp.echg(0, 95) # N point of MAPNET set to 95. Default 100
inp.echg.coordina([-4., -4., 0.], [4, -4, 0.], [4., 4., 0.])
inp.echg.margins(1.5, 1.5, 1.5, 1.5)
inp.echg.rectangu()

# BAND
inp.band('Band structure', 3, 4, 200, 1, 26, 1, 0,
         [[[0, 0, 0], [2, 0, 0]], # 3*2*3 list. 3 line segments, 2 ending points each segment and 3 coordinates each point
          [[2, 0, 0], [2, 2, 2]],
          [[2, 2, 2], [1, 0, 2]]])

# Write the input
inp.write_file('data/prop_input_1.d3')

The d3 formatted string can be examined by calling the `.data` attribute. The 'BAND' block is printed before NEWK even if it is set after NEWK, because the BAND calculation overwrites NEWK, which, in some circumstances, might lead to error, for example, 'NEWK-BAND-DOSS'. In CRYSTALpytools, BAND block is always printed before NEWK unless an appended sub-block is added (see below, charge difference section).

In [3]:
print(inp.data)

BAND
Band structure 
3 4 200 1 26 1 0 
0  0  0    2  0  0    
2  0  0    2  2  2    
2  2  2    1  0  2    
NEWK
12 24 
1 0 
ECHG
0
95
COORDINA
-4.000000 -4.000000  0.000000  
 4.000000 -4.000000  0.000000  
 4.000000  4.000000  0.000000  
RECTANGU
MARGINS
1.5 1.5 1.5 1.5 
END
END



#### Bands from pymatgen HighSymmKpath object

In this example, a band calculation is generated by the shortcut function `make_bands_block()`. A pymatgen HighSymmKpath object can be called.

In [4]:
from pymatgen.symmetry.bandstructure import HighSymmKpath
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

from CRYSTALpytools.crystal_io import Crystal_output
from CRYSTALpytools.crystal_io import Properties_input
from CRYSTALpytools.convert import cry_out2pmg


# Create the bands input object
inp = Properties_input()

# Read the structure
mgo = cry_out2pmg('data/Mg2O2_O1_100_1.out')
mgo_prim = SpacegroupAnalyzer(mgo).get_primitive_standard_structure(international_monoclinic=False)

# Obtain the k path object
k_path = HighSymmKpath(mgo_prim)

inp.make_bands_block(k_path, 200, 1, 26)

# Write the input
inp.write_file('data/prop_input_2.d3')
print(inp.data)

BAND
BAND STRUCTURE CALCULATION 
10 2 200 1 26 1 0 
0  0  0    0  1  0    
0  1  0    1  1  0    
1  1  0    0  0  0    
0  0  0    0  0  1    
0  0  1    0  1  1    
0  1  1    1  1  1    
1  1  1    0  0  1    
0  0  1    0  1  0    
0  1  0    0  1  1    
0  1  1    1  1  0    
END



#### DOSS and projected DOSS

In this example, a projected DOSS block is generated by the shortcut function `make_doss_block()`, which projects DOS onto all Mg atoms. The DOSS block is followed by a manually set 'DOSS-like' (COOP, COHP) block as currently no shortcut is provided.

Shortcut functions do not conflict with each other. The doss block is appended to the previous input.

In [5]:
# NEWK
inp.newk(12, 24, 1, 0)

# DOSS, with projection to all Mg atoms. An output file from crystal calculation is required
inp.make_doss_block(band_range=[1, 26], projections=['Mg'],
                    output_file='data/Mg2O2_O1_100_1.out')

# COOP, with projection to atom 1
inp.coop(1, 200, 1, 26, 2, 12, 1, [[-1, 1]])

# Write the input and print it
inp.write_file('data/prop_input_2.d3')
print(inp.data)

BAND
BAND STRUCTURE CALCULATION 
10 2 200 1 26 1 0 
0  0  0    0  1  0    
0  1  0    1  1  0    
1  1  0    0  0  0    
0  0  0    0  0  1    
0  0  1    0  1  1    
0  1  1    1  1  1    
1  1  1    0  0  1    
0  0  1    0  1  0    
0  1  0    0  1  1    
0  1  1    1  1  0    
NEWK
12 24 
1 0 
DOSS
1 200 1 26 2 12 1 
-2 1 2 
COOP
1 200 1 26 2 12 1 
-1 1 
END



#### Plot charge difference map

All the input block objects defined in CRYSTALpytools, including `Crystal_input` and `Properties_input`, are inherited from the same `BlockBASE` object defined in 'base.inputbase' script. Repeating the same keyword in the same block object overwrites the previous entry, so in principle the same keyword should appear only once in the same block. This leads to issues when running charge difference map, where 'ECHG' is defined twice with the second time a 'PATO' keyword is inserted.

To solve this problem, an `append` subblock can be added. Keywords for the initial calculation is set by `obj.keyword()` while keywords for the restarted calculation is set by `obj.append1.keyword()`. `Properties_input` objects alows at most 5 restarted calculations, i.e., `append1` to `append5`.

In [6]:
from CRYSTALpytools.crystal_io import Properties_input

# Initialization
inp = Properties_input()

# Initial calculation
inp.echg(0) # Default N point MAPNET = 100
inp.echg.coordina([-4., -4., 0.], [4, -4, 0.], [4., 4., 0.])
inp.echg.margins(1.5, 1.5, 1.5, 1.5)
inp.echg.rectangu()

# Reference calculation
inp.append1.pato(0, 0)
inp.append1.echg(0)
inp.append1.echg.coordina([-4., -4., 0.], [4, -4, 0.], [4., 4., 0.])
inp.append1.echg.margins(1.5, 1.5, 1.5, 1.5)
inp.append1.echg.rectangu()

# Write the input and print it
inp.write_file('data/prop_input_3.d3')
print(inp.data)

ECHG
0
100
COORDINA
-4.000000 -4.000000  0.000000  
 4.000000 -4.000000  0.000000  
 4.000000  4.000000  0.000000  
RECTANGU
MARGINS
1.5 1.5 1.5 1.5 
END
PATO
0 0 
ECHG
0
100
COORDINA
-4.000000 -4.000000  0.000000  
 4.000000 -4.000000  0.000000  
 4.000000  4.000000  0.000000  
RECTANGU
MARGINS
1.5 1.5 1.5 1.5 
END
END



### Properties_output class

In [1]:
from CRYSTALpytools.crystal_io import Properties_output

#### Band structure
Either 'BAND.DAT' or 'fort.25' format can be used.

In [2]:
mgo_bands = Properties_output()
mgo_bands.read_electron_band('data/mgo_BAND_dat.BAND')
mgo_bands

/tmp/ipykernel_18529/1396661211.py:2: UserWarning: Properties output file not found: 3D k path not available
  mgo_bands.read_electron_band('data/mgo_BAND_dat.BAND')


#### Density of states (DOS)
Either 'DOSS.DAT' or 'fort.25' format can be used.

In [3]:
mgo_doss = Properties_output()
mgo_doss = mgo_doss.read_electron_dos('data/mgo_DOSS_dat.DOSS')
mgo_doss

### Crystal_gui class

In [34]:
from CRYSTALpytools.crystal_io import Crystal_gui

#### Read CRYSTAL GUI file
A gui file is read by `Crystal_gui().read_gui()` method. The file can be converted into other geometry data formats including Pymatgen Structure/Molecule object and CIF file using functions in `CRYSTALpytools.convert`.

In [36]:
gui = Crystal_gui().read_gui('data/mgo_optgeom.opttraj/mgo-opt004.gui')
print('Dimensionality = {:d}\n'.format(gui.dimensionality))
print('Space group = {:d}\n'.format(gui.space_group))
print('N atoms = {:d}\n'.format(gui.n_atoms))
print('N symmetry operations = {:d}\n'.format(gui.n_symmops))

Dimensionality = 3

Space group = 225

N atoms = 2

N symmetry operations = 48



#### Write data as CRYSTAL GUI file
Other geometry data formats including Pymatgen Structure/Molecule object and CIF can be read and converted into CRYSTAL gui file.

* 3D structure

A conventional MgO cell is generated here, which will not be reduced to ensure consistency. To get the irreducible representation, use `CRYSTALpytools.geometry.refine_geometry()` function

In [37]:
from CRYSTALpytools.convert import cry_pmg2gui
from pymatgen.core import Structure, Lattice

# A conventional cell is generated here, which will not be reduced to maintain consistency
substrate = Structure.from_spacegroup("Fm-3m", Lattice.cubic(4.217), ["Mg",'O'], [[0, 0, 0],[0.5,0.5,0.5]])
mgo_gui = cry_pmg2gui(substrate, gui_file=None)
mgo_gui.write_gui('data/mgo_write_gui.gui')

In [38]:
! cat data/mgo_write_gui.gui

   3   1   1
  4.217000000000E+00  0.000000000000E+00  0.000000000000E+00
  0.000000000000E+00  4.217000000000E+00  0.000000000000E+00
  0.000000000000E+00  0.000000000000E+00  4.217000000000E+00
   48
      1.000000000000      0.000000000000      0.000000000000
      0.000000000000      1.000000000000      0.000000000000
      0.000000000000      0.000000000000      1.000000000000
      0.000000000000      0.000000000000      0.000000000000
     -1.000000000000      0.000000000000      0.000000000000
      0.000000000000     -1.000000000000      0.000000000000
      0.000000000000      0.000000000000     -1.000000000000
      0.000000000000      0.000000000000      0.000000000000
      0.000000000000     -1.000000000000      0.000000000000
      1.000000000000      0.000000000000      0.000000000000
      0.000000000000      0.000000000000      1.000000000000
      0.000000000000      0.000000000000      0.000000000000
      0.000000000000      1.000000000000      0.000000000000
     

* 2D slab

A MgO (100) slab is generated. The thickness of vacuum layer (which is not very meaningful since CRYSTAL is based on atomic orbitals) is set to be 25 Angstrom, instead of the default of 500 Angstrom in z direction. 

In [39]:
from pymatgen.core import Structure, Lattice
from pymatgen.core.surface import SlabGenerator

#Generate bulk structure
substrate = Structure.from_spacegroup("Fm-3m", Lattice.cubic(4.217), ["Mg",'O'], [[0, 0, 0],[0.5,0.5,0.5]])
#Generate slab
substrate = SlabGenerator(substrate, (1,0,0), 5., 10., center_slab=True).get_slab()

# Pymatgen get_slab will return to a 3D structure. Use pbc to force the
# implementation of 2D periodicity otherwise the vaccum input is invalid
mgo_slab_gui = cry_pmg2gui(substrate, gui_file=None, pbc=[True, True, False], vacuum=25)
mgo_slab_gui.write_gui('data/mgo_100_write_gui.gui',mgo_slab_gui)

In [40]:
! cat data/mgo_100_write_gui.gui

   2   1   1
  2.981869296264E+00  0.000000000000E+00  0.000000000000E+00
  0.000000000000E+00  2.981869296264E+00  0.000000000000E+00
  0.000000000000E+00  0.000000000000E+00  3.132550000000E+01
   16
      1.000000000000      0.000000000000      0.000000000000
      0.000000000000      1.000000000000      0.000000000000
     -0.000000000000     -0.000000000000      1.000000000000
      0.000000000000      0.000000000000      0.000000000000
      0.000000000000     -1.000000000000      0.000000000000
      1.000000000000     -0.000000000000      0.000000000000
     -0.000000000000     -0.000000000000      1.000000000000
      0.000000000000      0.000000000000      0.000000000000
     -1.000000000000     -0.000000000000      0.000000000000
      0.000000000000     -1.000000000000      0.000000000000
     -0.000000000000     -0.000000000000      1.000000000000
      0.000000000000      0.000000000000      0.000000000000
     -0.000000000000      1.000000000000      0.000000000000
     